In [61]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import re
import pymongo
from pymongo import MongoClient


In [55]:
# Helper Function
def find_tag_value(soup, tag, class_name, index=None):
    try:
        if index is None:
            return soup.find(tag, class_=(class_name)).text.strip()
        else:
            return soup.find_all(tag, class_=(class_name))[index].text.strip()
    except AttributeError:
        return None

    
def find_tag_attr(job_card, attr):
    try:
        return job_card.find('div', {'data-gtm-job-' + attr: True})['data-gtm-job-' + attr]
    except (AttributeError, KeyError):
        return None
    

In [56]:
# Initialization
data = []
page = 1

# Find using Computer Information Technology

In [57]:
while True:
    url = 'https://glints.com/id/job-category/computer-information-technology?page={}'.format(page)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')
    job_cards = soup.find_all('div', class_='JobCardsc__JobcardContainer-sc-hmqj50-0 iirqVR CompactOpportunityCardsc__CompactJobCardWrapper-sc-dkg8my-2 bMyejJ compact_job_card')
    if len(job_cards) == 0:
        print('No More Jobs')
        break
    print('page',page,'jobs found', len(job_cards))
            
    for job_card in job_cards:
        job_title = find_tag_value(job_card, 'h3', 'CompactOpportunityCardsc__JobTitle-sc-dkg8my-9 hgMGcy')
        company_name = find_tag_value(job_card, 'a', 'CompactOpportunityCardsc__CompanyLink-sc-dkg8my-10 iTRLWx')
        job_location = find_tag_value(job_card, 'span', 'CardJobLocation__StyledTruncatedLocation-sc-1by41tq-1 kEinQH')
        work_place = find_tag_value(job_card, 'div', 'TagStyle-sc-r1wv7a-4 bJWZOt CompactOpportunityCardTags__Tag-sc-610p59-1 hncMah')
        # job_tags = job_cards.find_all('div', class_='TagStyle__TagContentWrapper-sc-r1wv7a-1 koGVuk')
        job_tags = [find_tag_value(job_card, 'div', 'TagStyle-sc-r1wv7a-4 bJWZOt CompactOpportunityCardTags__Tag-sc-610p59-1 hncMah', index=i) for i in range(4)]
        job_tag_values = [tag for tag in job_tags]
        years_experience = job_tag_values[2]
        min_education = job_tag_values[3]
        
        salary_range = find_tag_value(job_card, 'span', 'CompactOpportunityCardsc__NotDisclosedMessage-sc-dkg8my-23 hivaYx')
        if salary_range:
            salary_range = 'Not Written'
        else:
            salary_range = find_tag_value(job_card, 'span', 'CompactOpportunityCardsc__SalaryWrapper-sc-dkg8my-29 gfPeyg')
        
        skill = find_tag_attr(job_card, 'card-info').replace('experience,logo,','')
        job_id = find_tag_attr(job_card, 'id')
        work_type = find_tag_attr(job_card, 'type')
        category = find_tag_attr(job_card, 'category')
        sub_category = find_tag_attr(job_card, 'sub-category')
        role = find_tag_attr(job_card, 'role')

        more_detail_link = 'https://glints.com'+ job_card.find('a', class_= 'CompactOpportunityCardsc__CardAnchorWrapper-sc-dkg8my-24 knEIai job-search-results_job-card_link').get('href')
        # detail link request
        response_detail = requests.get(more_detail_link, headers=headers)
        soup_detail = BeautifulSoup(response_detail.content, 'html.parser')
        # posted_date = find_tag_value(soup_detail,'div','JobDescriptionsc__DescriptionContainer-sc-22zrgx-2 jCwTA-d')
        # print('posted_date:',posted_date)
        job_description = find_tag_value(soup_detail,'div','JobDescriptionsc__DescriptionContainer-sc-22zrgx-2 jCwTA-d')
        
        data.append({
            'job_title': job_title,
            'company_name': company_name,
            'job_location': job_location,
            'work_place': work_place,
            'salary_range': salary_range,
            'more_detail_link': more_detail_link,
            'job_id': job_id,
            'work_type': work_type,
            'workplace': work_place,
            'years_experience': years_experience,
            'min_education': min_education,
            'category': category,
            'sub_category': sub_category,
            'role': role
        })
        
    page += 1
    
# stop the loop
print('Total Jobs:', len(data))
df = pd.DataFrame(data)


page 1 jobs found 30
page 2 jobs found 30
page 3 jobs found 30
page 4 jobs found 30
page 5 jobs found 30
page 6 jobs found 30
page 7 jobs found 30
page 8 jobs found 30
page 9 jobs found 30
page 10 jobs found 30
page 11 jobs found 30
page 12 jobs found 30
page 13 jobs found 30
page 14 jobs found 30
page 15 jobs found 30
page 16 jobs found 30
page 17 jobs found 30
page 18 jobs found 30
page 19 jobs found 30
page 20 jobs found 30
page 21 jobs found 30
page 22 jobs found 30
page 23 jobs found 30
page 24 jobs found 30
page 25 jobs found 30
page 26 jobs found 30
page 27 jobs found 30
page 28 jobs found 30
page 29 jobs found 30
page 30 jobs found 30
page 31 jobs found 30
page 32 jobs found 30
page 33 jobs found 30
page 34 jobs found 30
No More Jobs
Total Jobs: 1020


In [63]:
df = pd.DataFrame(data)

In [59]:
# drop duplicate keep first
df = df.drop_duplicates(subset='job_id', keep='first')

In [60]:
df['company_name'] = df['company_name'].str.upper()

In [ ]:
data_dict = df.to_dict("records")

client = MongoClient('mongodb://localhost:27017/')
db = client['jobs']
collection = db['glints']

collection.insert_many(data_dict)

cursor = collection.find({})
for document in cursor:
    print(document)


In [75]:
from pymongo import MongoClient

# Connection URI
uri = 'mongodb://localhost:27017'

# Database Name
db_name = 'jobs'

# Collection Name
collection_name = 'glints'

# Create a new MongoClient
client = MongoClient(uri)

try:
    # Connect to the database
    db = client[db_name]

    # Access the collection
    collection = db[collection_name]

    # Aggregation pipeline
    pipeline = [
        {
            '$group': {
                '_id': '$sub_category',
                'count': {'$sum': 1}
            }
        }
    ]

    # Run the aggregation query
    result = list(collection.aggregate(pipeline))

    # Print the result
    for doc in result:
        print(f"{doc['_id']}: {doc['count']}")

finally:
    # Close the connection
    client.close()


Mobile Development: 44
Operation, Maintenance & Technical Support: 155
Senior Technical Position: 10
Artificial Intelligence: 26
Technical Sales Support: 16
Data: 79
Frontend Development: 41
Other Computer & Information Technology: 110
Electronic & Semiconductor: 51
Hardware Development: 16
Network, System & Database Admin: 64
Software: 272
QA & Software Testing: 31
Backend Development: 86
Telecommunication: 18


# Find using Listed Job Search

In [68]:
list_jobs = {'golang','.net','php developer','laravel','java','python','nodejs','reactjs','nextjs','angularjs','fluter','kotlin','vuejs','backend','frontend','mobile','data analyst','data scientist','data engineer','software engineer','software developer','full-stack','programmer','javascript','user interface','user experience','hr officer','accounting officer'}


page = 1
data = []
list_jobs = {'php developer'}
for i in list_jobs:
    search_position = i
    search_position = search_position.lower().replace(' ','+')

# template = 'https://glints.com/id/opportunities/jobs/explore?'
# url_params = 'keyword={}&country=ID&locationName={}' if search_position and location else 'keyword={}&country=ID&locationName=All+Cities%2FProvinces'

    while True:
        url = 'https://glints.com/id/opportunities/jobs/explore?keyword={}&country=ID&locationName=All+Cities%2FProvinces&page={}'.format(search_position,page)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)

        # if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        job_cards = soup.find_all('div', class_='JobCardsc__JobcardContainer-sc-hmqj50-0 iirqVR CompactOpportunityCardsc__CompactJobCardWrapper-sc-dkg8my-2 bMyejJ compact_job_card')
        if len(job_cards) == 0:
            print('No More Jobs')
            break
        print('page',page,'jobs found', len(job_cards))
                
        for job_card in job_cards:
            job_title = find_tag_value(job_card, 'h3', 'CompactOpportunityCardsc__JobTitle-sc-dkg8my-9 hgMGcy')
            company_name = find_tag_value(job_card, 'a', 'CompactOpportunityCardsc__CompanyLink-sc-dkg8my-10 iTRLWx')
            job_location = find_tag_value(job_card, 'span', 'CardJobLocation__StyledTruncatedLocation-sc-1by41tq-1 kEinQH')
            work_place = find_tag_value(job_card, 'div', 'TagStyle-sc-r1wv7a-4 bJWZOt CompactOpportunityCardTags__Tag-sc-610p59-1 hncMah')
            # job_tags = job_cards.find_all('div', class_='TagStyle__TagContentWrapper-sc-r1wv7a-1 koGVuk')
            
            try:
                job_tags = [find_tag_value(job_card, 'div', 'TagStyle-sc-r1wv7a-4 bJWZOt CompactOpportunityCardTags__Tag-sc-610p59-1 hncMah', index=i) for i in range(4)]
            except Exception as e:
                print(f"Error occurred while extracting job tags: {e}")

            job_tag_values = [tag for tag in job_tags]
            years_experience = job_tag_values[2] if len(job_tag_values) > 2 else None
            min_education = job_tag_values[3] if len(job_tag_values) > 3 else None

            # job_tags = [find_tag_value(job_card, 'div', 'TagStyle-sc-r1wv7a-4 bJWZOt CompactOpportunityCardTags__Tag-sc-610p59-1 hncMah', index=i) for i in range(4)]
            # job_tag_values = [tag for tag in job_tags]
            # years_experience = job_tag_values[2]
            # min_education = job_tag_values[3]
            
            salary_range = find_tag_value(job_card, 'span', 'CompactOpportunityCardsc__NotDisclosedMessage-sc-dkg8my-23 hivaYx')
            if salary_range:
                salary_range = 'Not Written'
            else:
                salary_range = find_tag_value(job_card, 'span', 'CompactOpportunityCardsc__SalaryWrapper-sc-dkg8my-29 gfPeyg')
            
            skill = find_tag_attr(job_card, 'card-info').replace('experience,logo,','')
            job_id = find_tag_attr(job_card, 'id')
            work_type = find_tag_attr(job_card, 'type')
            category = find_tag_attr(job_card, 'category')
            sub_category = find_tag_attr(job_card, 'sub-category')
            role = find_tag_attr(job_card, 'role')

            more_detail_link = 'https://glints.com'+ job_card.find('a', class_= 'CompactOpportunityCardsc__CardAnchorWrapper-sc-dkg8my-24 knEIai job-search-results_job-card_link').get('href')
            # detail link request
            response_detail = requests.get(more_detail_link, headers=headers)
            soup_detail = BeautifulSoup(response_detail.content, 'html.parser')
            # posted_date = find_tag_value(soup_detail,'div','JobDescriptionsc__DescriptionContainer-sc-22zrgx-2 jCwTA-d')
            # print('posted_date:',posted_date)
            job_description = find_tag_value(soup_detail,'div','JobDescriptionsc__DescriptionContainer-sc-22zrgx-2 jCwTA-d')
            
            data.append({
                'job_title': job_title,
                'company_name': company_name,
                'job_location': job_location,
                'work_place': work_place,
                'salary_range': salary_range,
                'more_detail_link': more_detail_link,
                'job_id': job_id,
                'work_type': work_type,
                'workplace': work_place,
                'years_experience': years_experience,
                'min_education': min_education,
                'category': category,
                'sub_category': sub_category,
                'role': role
            })
            
        page += 1
        
    break
    

page 1 jobs found 30
page 2 jobs found 30
page 3 jobs found 30
page 4 jobs found 30
page 5 jobs found 30
page 6 jobs found 30
page 7 jobs found 30
Error occurred while extracting job tags: list index out of range
page 8 jobs found 30
page 9 jobs found 30
page 10 jobs found 30
page 11 jobs found 30
page 12 jobs found 30
page 13 jobs found 30
page 14 jobs found 30
page 15 jobs found 30
page 16 jobs found 30
page 17 jobs found 30
page 18 jobs found 30
page 19 jobs found 30
page 20 jobs found 30
Error occurred while extracting job tags: list index out of range
page 21 jobs found 30
page 22 jobs found 3
No More Jobs


In [69]:
df2 = pd.DataFrame(data)

In [71]:
df2.to_csv('php.csv', index=False)

In [ ]:
data_dict = df.to_dict("records")

client = MongoClient('mongodb://localhost:27017/')
db = client['jobs']
collection = db['glints']

collection.insert_many(data_dict)

cursor = collection.find({})
for document in cursor:
    print(document)


# Merging 2 CSV Files

In [15]:
df1 = pd.read_csv('list_of_jobs_glints.csv')
df2 = pd.read_csv('list_of_jobs_glints_2.csv')

In [16]:

merged_df = pd.concat([df, df2])
merged_df.to_csv('merged_file.csv', index=False)


# Cleaning Merged File 

In [17]:
# remove duplicate based on job_id
merged_df = merged_df.drop_duplicates(subset='job_id', keep='first')

In [ ]:
merged_df.to_csv('merged_file.csv', index=False)

In [19]:
merged_file = pd.read_csv('merged_file.csv')

## Cleaning Company Name Column

In [20]:
def clean_column(job_dataset, column_name):
    corpus = []
    for i in range(0, len(job_dataset)):
        text = re.sub(r'[."]', '', job_dataset[column_name][i])
        text = text.upper()
        text = text.split()
        text = ' '.join(text)
        corpus.append(text)

    job_dataset[column_name] = corpus
    return job_dataset

In [21]:
clean_column(merged_file,'company_name')

,job_title,company_name,job_location,work_place,salary_range,more_detail_link,job_id,work_type,workplace,years_experience,min_education,category,sub_category,role
0,Back End Developer,PT RECT MEDIA KOMPUTINDO,"Semarang Tengah, Semarang, Jawa Tengah",Kerja di kantor,Rp3jt-Rp5jt,https://glints.com/id/opportunities/jobs/back-...,fbcc6efd-3e92-47c8-848b-28fd57ee1478,full_time,Kerja di kantor,1 – 3 tahun,Minimal Sarjana (S1),Computer & Information Technology,Backend Development,Backend Developer
1,IT Security Operation,QODE WORLD,"Setiabudi, Jakarta Selatan, DKI Jakarta",Kerja di kantor,Not Written,https://glints.com/id/opportunities/jobs/it-se...,854b311f-56b0-436b-8d45-71f4ebf588ef,full_time,Kerja di kantor,3 – 5 tahun,Minimal Sarjana (S1),Computer & Information Technology,"Operation, Maintenance & Technical Support",System Security
2,Experience IT EDC Android Developer,PCS PAYMENT,"Setiabudi, Jakarta Selatan, DKI Jakarta",Kerja di kantor,"Rp6,5jt-Rp9,5jt",https://glints.com/id/opportunities/jobs/exper...,d71afe9e-cdbc-4eae-8e45-eab093105b49,full_time,Kerja di kantor,3 – 5 tahun,Minimal Sarjana (S1),Computer & Information Technology,Mobile Development,Android Developer
3,Programmer Project Based,PT DAVID ROY INDONESIA,"Kalideres, Jakarta Barat, DKI Jakarta",Kerja di kantor,Not Written,https://glints.com/id/opportunities/jobs/progr...,46f2b84c-fa9d-403d-8ff3-3cfd16d91580,contract,Kerja di kantor,Kurang dari setahun,Minimal Sarjana (S1),Computer & Information Technology,Software,Programmer
4,IT Frontend & Backend Developer,PT LOBUNTA KENCANA RAYA,"Sawah Besar, Jakarta Pusat, DKI Jakarta",Kerja di kantor,Not Written,https://glints.com/id/opportunities/jobs/it-fr...,92139ff9-a5bc-49aa-a319-398def61398d,full_time,Kerja di kantor,1 – 3 tahun,Minimal Sarjana (S1),Computer & Information Technology,Software,Web Developer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,Social Media Officer,ATLAS BEACH FEST BALI,"Kuta Utara, Badung, Bali",Kerja di kantor,Not Written,https://glints.com/id/opportunities/jobs/socia...,8cf870be-78c3-4402-a8f8-0c7b2f4313f4,full_time,NaN,NaN,NaN,Marketing,Digital Marketing,Social Media Specialist
1046,Guest Relationship Officer,AKMANI HOTEL JAKARTA,"Menteng, Jakarta Pusat, DKI Jakarta",Kerja di kantor,Not Written,https://glints.com/id/opportunities/jobs/guest...,2bf76628-31f6-44ce-87c4-04ef92b632dd,full_time,NaN,NaN,NaN,Administrative & Human Resources,Administrative,Receptionist
1047,Strategic Field Officer,LPK WIWITAN BARU,"Antapani, Bandung, Jawa Barat",Kerja di kantor,Not Written,https://glints.com/id/opportunities/jobs/strat...,c7cce9de-c248-4783-9eb2-03c558e069ce,full_time,NaN,NaN,NaN,Business Development & Sales,Sales,Sales Operations
1048,Mobile Marketer,MBOK BEREK NY SUBEKTI,"Cipayung, Jakarta Timur, DKI Jakarta",Kerja di kantor,"Rp3,5jt-Rp5jt",https://glints.com/id/opportunities/jobs/mobil...,96be39df-c5e7-47f3-aa3d-f75fdf132d43,full_time,NaN,NaN,NaN,Business Development & Sales,Other Business Development & Sales,Other Business Development & Sales Positions


In [22]:
merged_file.isnull().sum()

job_title              0
company_name           0
job_location           0
work_place             0
salary_range           0
more_detail_link       0
job_id                 0
work_type              0
workplace           1020
years_experience    1020
min_education       1020
category               0
sub_category           0
role                   0
dtype: int64

In [ ]:
merged_file['role'].value_counts().to_csv('role.csv')

In [ ]:
merged_file['sub_category'].value_counts().to_csv('sub_cat.csv')